In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from rdkit import Chem
from IPython.core.display import display, HTML

In [18]:
def force_show_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(HTML(df.to_html()))

# BindingDB

In [19]:
bindingDB = pd.read_csv('..\Raw_data\Binding_DB\CEC293F933A98F82D59BD6FBD763B2F4ki.tsv', sep = '\t')
bindingDB = bindingDB[['Ligand SMILES', 'IC50 (nM)']]
bindingDB['database_label'] = 'BindingDB'
force_show_all(bindingDB)

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,0.800000,BindingDB
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@@H]2N)CC1,0.800,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(O)nc4CCCCn4c3=O)c2Cl)c(N)n1,1.000000,BindingDB
6,N[C@H]1c2cccnc2OC11CCN(CC1)c1nc(N)c(-c2cccc(Cl)c2Cl)c(n1)C(N)=O,1.000,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C(Cl)=CCC2)c(n1)C(N)=O,1.000,BindingDB
8,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)c2Cl)c2nccn12,1.20,BindingDB


In [20]:
bindingDB.shape

(2189, 3)

## Missing Values

In [21]:
bindingDB.isnull().sum()

Ligand SMILES      0
IC50 (nM)         31
database_label     0
dtype: int64

In [22]:
bindingDB = bindingDB.dropna()

In [23]:
bindingDB.shape

(2158, 3)

## BindingDB Data

In [24]:
bindingDB_data = bindingDB
bindingDB_data

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800000,BindingDB
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1...,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-...,1.000,BindingDB
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1...,1.000,BindingDB
...,...,...,...
2184,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(N)c4)...,37.0,BindingDB
2185,Cn1c(ncc(Sc2cccc(Cl)c2Cl)c1=O)N1CCC2(COC[C@H]2...,37.0,BindingDB
2186,Cn1c(nc2[nH]nc(-c3ccccc3)c2c1=O)N1CCC2(C[C@H](...,37.0,BindingDB
2187,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C...,37,BindingDB


## Number of duplicate molecules

In [25]:
bindingDB_data.iloc[:, 0].duplicated(keep=False).sum()

670

## Analysis of Duplicate molecules

In [26]:
duplicates = bindingDB_data.loc[bindingDB_data.iloc[:, 0].duplicated(keep=False), :]
duplicates.shape

(670, 3)

In [27]:
force_show_all(duplicates)

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,0.800000,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
6,N[C@H]1c2cccnc2OC11CCN(CC1)c1nc(N)c(-c2cccc(Cl)c2Cl)c(n1)C(N)=O,1.000,BindingDB
8,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
10,Cc1nc(N2CCC3(CC2)Oc2ccccc2[C@H]3N)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.40,BindingDB
11,Cc1nc(N2CCC3(CC2)Oc2ccccc2[C@H]3N)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.4,BindingDB
12,C[C@@H]1OCC2(CCN(CC2)c2nc(C)c(nc2CO)-c2cccc(Cl)c2Cl)[C@@H]1N,1.5,BindingDB
17,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1ccc(F)c(C)c1Cl,2.0,BindingDB


In [28]:
sorted_duplicates = duplicates.sort_values(by=['IC50 (nM)'], ascending=True)
force_show_all(sorted_duplicates)

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,0.800000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
6,N[C@H]1c2cccnc2OC11CCN(CC1)c1nc(N)c(-c2cccc(Cl)c2Cl)c(n1)C(N)=O,1.000,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
8,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
11,Cc1nc(N2CCC3(CC2)Oc2ccccc2[C@H]3N)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.4,BindingDB
10,Cc1nc(N2CCC3(CC2)Oc2ccccc2[C@H]3N)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.40,BindingDB
12,C[C@@H]1OCC2(CCN(CC2)c2nc(C)c(nc2CO)-c2cccc(Cl)c2Cl)[C@@H]1N,1.5,BindingDB
1949,C[C@@H]1OCC2(CCN(CC2)c2nc(N)c(Sc3ccnc(C4CC4)c3Cl)c(=O)n2C)[C@@H]1N,10.0,BindingDB


In [29]:
def show_duplicate_rows(smile):
    df = sorted_duplicates[sorted_duplicates['Ligand SMILES']==smile]
    force_show_all(df)

In [30]:
show_duplicate_rows('Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1')

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,0.800000,BindingDB
422,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,270,BindingDB
594,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,5.0,BindingDB


In [31]:
show_duplicate_rows('Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl')

,Ligand SMILES,IC50 (nM),database_label
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
595,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,5.0,BindingDB


In [32]:
# dropping duplicate values
unique_in_duplicates = duplicates.drop_duplicates(subset ="Ligand SMILES" ,keep='last')
unique_in_duplicates

,Ligand SMILES,IC50 (nM),database_label
111,CC1(CN)CCN(CC1)c1ncc(Sc2cccc(Cl)c2Cl)[nH]c1=O,<50,BindingDB
182,NCC1(CCN(CC1)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl)c1cc...,65.0,BindingDB
183,NC1C2CN(CC12)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl,65.0,BindingDB
199,NC[C@H]1CCN(C1)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl,68.0,BindingDB
235,CCCC1(CN)CCN(CC1)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl,75.0,BindingDB
...,...,...,...
2170,Cn1c(nc(N)c(Sc2cccnc2C(F)(F)F)c1=O)N1CCC2(COC[...,34.0,BindingDB
2171,Cn1c(nc2[nH]nc(-c3cccc(Cl)c3Cl)c2c1=O)N1CCC(C)...,34,BindingDB
2174,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,35.0,BindingDB
2180,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB


In [33]:
unique_in_duplicates.shape

(250, 3)

In [34]:
unique_in_duplicates.iloc[:, 0].duplicated().sum()

0

In [35]:
for smile in unique_in_duplicates['Ligand SMILES'].values:
    show_duplicate_rows(smile)
    print("******************************")

,Ligand SMILES,IC50 (nM),database_label
106,CC1(CN)CCN(CC1)c1ncc(Sc2cccc(Cl)c2Cl)[nH]c1=O,<50,BindingDB
111,CC1(CN)CCN(CC1)c1ncc(Sc2cccc(Cl)c2Cl)[nH]c1=O,<50,BindingDB


******************************


,Ligand SMILES,IC50 (nM),database_label
182,NCC1(CCN(CC1)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl)c1ccccc1,65.0,BindingDB
179,NCC1(CCN(CC1)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl)c1ccccc1,65.0,BindingDB


******************************


,Ligand SMILES,IC50 (nM),database_label
183,NC1C2CN(CC12)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl,65.0,BindingDB
180,NC1C2CN(CC12)c1cnc(c(N)n1)-c1cccc(Cl)c1Cl,65.0,BindingDB


...

******************************


,Ligand SMILES,IC50 (nM),database_label
2177,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC5)c4Cl)c3c(=O)n2C)[C@@H]1N,36,BindingDB
2178,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC5)c4Cl)c3c(=O)n2C)[C@@H]1N,36,BindingDB
2179,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC5)c4Cl)c3c(=O)n2C)[C@@H]1N,36,BindingDB
2180,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC5)c4Cl)c3c(=O)n2C)[C@@H]1N,36,BindingDB
2176,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC5)c4Cl)c3c(=O)n2C)[C@@H]1N,36.0,BindingDB


******************************


,Ligand SMILES,IC50 (nM),database_label
2187,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C)c3c(=O)n2C)[C@@H]1N,37,BindingDB
2188,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C)c3c(=O)n2C)[C@@H]1N,37,BindingDB
50,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C)c3c(=O)n2C)[C@@H]1N,37.0,BindingDB
56,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C)c3c(=O)n2C)[C@@H]1N,38,BindingDB
52,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C)c3c(=O)n2C)[C@@H]1N,38,BindingDB


******************************


In [37]:
not_duplicates = bindingDB_data.loc[~bindingDB_data.iloc[:, 0].duplicated(), :]
not_duplicates.shape

(1738, 3)

In [38]:
force_show_all(not_duplicates)

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@H]2N)CC1,0.800000,BindingDB
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3[C@@H]2N)CC1,0.800,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.000,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(O)nc4CCCCn4c3=O)c2Cl)c(N)n1,1.000000,BindingDB
6,N[C@H]1c2cccnc2OC11CCN(CC1)c1nc(N)c(-c2cccc(Cl)c2Cl)c(n1)C(N)=O,1.000,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C(Cl)=CCC2)c(n1)C(N)=O,1.000,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)c2Cl)c2nccn12,1.20,BindingDB
10,Cc1nc(N2CCC3(CC2)Oc2ccccc2[C@H]3N)c(C)c(=O)n1-c1cccc(Cl)c1Cl,1.40,BindingDB
12,C[C@@H]1OCC2(CCN(CC2)c2nc(C)c(nc2CO)-c2cccc(Cl)c2Cl)[C@@H]1N,1.5,BindingDB


In [39]:
not_duplicates.iloc[:, 0].duplicated().sum()

0

In [40]:
unique_df = pd.concat([not_duplicates, duplicates])
unique_df

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800000,BindingDB
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1...,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-...,1.000,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,1.000000,BindingDB
...,...,...,...
2178,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2179,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2180,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2187,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C...,37,BindingDB


In [41]:
unique_df.shape

(2408, 3)

In [42]:
unique_df.iloc[:, 0].duplicated().sum()

670

In [43]:
duplicates_uni = unique_df.loc[unique_df.iloc[:, 0].duplicated(), :]
duplicates_uni

,Ligand SMILES,IC50 (nM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800000,BindingDB
2,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1...,1.000000,BindingDB
3,Cc1nc(N2CCC3(Cc4ccccc4[C@H]3N)CC2)c(C)c(=O)n1-...,1.000,BindingDB
4,Cc1nc(N2CCC3(Cc4ncccc4[C@@H]3N)CC2)c(C)c(=O)n1...,1.000,BindingDB
6,N[C@H]1c2cccnc2OC11CCN(CC1)c1nc(N)c(-c2cccc(Cl...,1.000,BindingDB
...,...,...,...
2178,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2179,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2180,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(NC5CC...,36,BindingDB
2187,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccc(N)nc4C...,37,BindingDB


In [44]:
# dropping duplicate values
unique_df.drop_duplicates(subset ="Ligand SMILES" ,keep='first',inplace=True)
unique_df.shape

(1738, 3)

## Removing duplicate molecules

In [45]:
# dropping duplicate values
bindingDB_data.drop_duplicates(subset ="Ligand SMILES" ,keep='last',inplace=True)
bindingDB_data

,Ligand SMILES,IC50 (nM),database_label
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.800,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,1.000000,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,1.000,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,1.20,BindingDB
13,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,1.50,BindingDB
...,...,...,...
2183,C[C@H]1OCC2(CCN(CC2)c2cc(=O)n(c(C)n2)-c2cccc(C...,37.0,BindingDB
2184,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(N)c4)...,37.0,BindingDB
2185,Cn1c(ncc(Sc2cccc(Cl)c2Cl)c1=O)N1CCC2(COC[C@H]2...,37.0,BindingDB
2186,Cn1c(nc2[nH]nc(-c3ccccc3)c2c1=O)N1CCC2(C[C@H](...,37.0,BindingDB


In [46]:
bindingDB_data.iloc[:, 0].duplicated().sum()

0

In [47]:
bindingDB_data.shape

(1738, 3)

In [48]:
def conversion_for_signs(s):
    if s[0] == '>' or s[0] == '<':
        return s[0] + str(float(s[1:])/1000)
    else:
        return str(float(s)/1000)

In [49]:
bindingDB_data['IC50 (nM)'] = bindingDB_data['IC50 (nM)'].apply(conversion_for_signs)
bindingDB_data

,Ligand SMILES,IC50 (nM),database_label
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
13,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
2183,C[C@H]1OCC2(CCN(CC2)c2cc(=O)n(c(C)n2)-c2cccc(C...,0.037,BindingDB
2184,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(N)c4)...,0.037,BindingDB
2185,Cn1c(ncc(Sc2cccc(Cl)c2Cl)c1=O)N1CCC2(COC[C@H]2...,0.037,BindingDB
2186,Cn1c(nc2[nH]nc(-c3ccccc3)c2c1=O)N1CCC2(C[C@H](...,0.037,BindingDB


In [50]:
bindingDB_data.columns = ['SMILES', 'IC50(microM)','database_label']
bindingDB_data

,SMILES,IC50(microM),database_label
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
13,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
2183,C[C@H]1OCC2(CCN(CC2)c2cc(=O)n(c(C)n2)-c2cccc(C...,0.037,BindingDB
2184,C[C@@H]1OCC2(CCN(CC2)c2nc3[nH]nc(-c4ccnc(N)c4)...,0.037,BindingDB
2185,Cn1c(ncc(Sc2cccc(Cl)c2Cl)c1=O)N1CCC2(COC[C@H]2...,0.037,BindingDB
2186,Cn1c(nc2[nH]nc(-c3ccccc3)c2c1=O)N1CCC2(C[C@H](...,0.037,BindingDB


# Review Papers

In [51]:
Literature_1_df = pd.read_csv('..\Raw_data\Literature\Literature_1.csv')
Literature_1_df  = Literature_1_df [['Ligand SMILES', 'IC50 (micro)']]
Literature_1_df .columns = ['SMILES', 'IC50(microM)']
Literature_1_df ['database_label'] = 'Literature_1'
Literature_1_df 

,SMILES,IC50(microM),database_label
0,O=C1C(O)=CC(C(O)=O)=CC2=CC=C(O)C(O)=C21,0.097,Literature_1
1,NC1=CC2=NSN=C2C=C1C#C,2.110,Literature_1
2,OC1=C(C(N2C3=CC=C(C=C3SC2=N1)OC)=O)CC4=CC=CC=C4,10.000,Literature_1
3,O=C(C=C)NC1=C2C=CC=CC2=NC=C1,35.000,Literature_1
4,O=C1N(C2=CCCN(C2)CC3=CC(C(/C=C/C4=CC=C(Cl)C=C4...,9.800,Literature_1
...,...,...,...
56,CCCCC/C=C\C/C=C\CCCCCCCCCCCCC(O)=O,23.900,Literature_1
57,OC1=CC(/C=C/C(O)=O)=CC=C1O,26.600,Literature_1
58,O[C@@H]([C@@H]([C@H]([C@H](CO)O1)O)O)C1OCC(O)C...,20.700,Literature_1
59,[H][C@]1(O[C@@]2(OC[C@H](C)CC2)[C@H]3C)C[C@@]4...,1.700,Literature_1


In [52]:
Literature_2_df  = pd.read_csv('..\Raw_data\Literature\Literature_2.csv')
Literature_2_df  = Literature_2_df [['Ligand SMILES', 'IC50 (microM)']]
Literature_2_df .columns = ['SMILES', 'IC50(microM)']
Literature_2_df ['database_label'] = 'Literature_2'
Literature_2_df 

,SMILES,IC50(microM),database_label
0,ClC1=C(Cl)C(C2=NC=C(N3CCC(N)(C)CC3)N=C2N)=CC=C1,0.07,Literature_2
1,ClC1=C(Cl)C(C2=C(N)N=C(N3C[C@H](C)N[C@H](C)C3)...,12.00,Literature_2
2,ClC1=CC=CC=C1CN2C3=NN=C(C4=C(O)C(OC)=CC=C4)N3C...,60.00,Literature_2
3,O=S(C1=CC=C2C(C=CC(/N=N/C3=CC(S(=O)(O)=O)=C(C=...,0.30,Literature_2
4,O=C1CC2=CC=C(S(NCC3=CC=C(F)C=C3)(=O)=O)C=C2/C1...,1.00,Literature_2
...,...,...,...
90,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
91,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
92,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
93,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [53]:
review_paper_data = pd.concat([Literature_1_df,Literature_2_df])
review_paper_data

,SMILES,IC50(microM),database_label
0,O=C1C(O)=CC(C(O)=O)=CC2=CC=C(O)C(O)=C21,0.097,Literature_1
1,NC1=CC2=NSN=C2C=C1C#C,2.110,Literature_1
2,OC1=C(C(N2C3=CC=C(C=C3SC2=N1)OC)=O)CC4=CC=CC=C4,10.000,Literature_1
3,O=C(C=C)NC1=C2C=CC=CC2=NC=C1,35.000,Literature_1
4,O=C1N(C2=CCCN(C2)CC3=CC(C(/C=C/C4=CC=C(Cl)C=C4...,9.800,Literature_1
...,...,...,...
90,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.640,Literature_2
91,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.580,Literature_2
92,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.470,Literature_2
93,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.360,Literature_2


In [ ]:
#review_paper_data[review_paper_data['SMILES']=='']

# Combined data = BindingDB + Review Paper

In [54]:
bindingDB_data['database_label'].value_counts()

BindingDB    1738
Name: database_label, dtype: int64

In [55]:
review_paper_data['database_label'].value_counts()

Literature_2    95
Literature_1    61
Name: database_label, dtype: int64

In [56]:
combined_data = pd.concat([bindingDB_data,review_paper_data])
combined_data

,SMILES,IC50(microM),database_label
1,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
5,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
7,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
9,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
13,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
90,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
91,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
92,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
93,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [57]:
combined_data = combined_data.reset_index()
combined_data = combined_data.drop(['index'],axis=1)
combined_data

,SMILES,IC50(microM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
1889,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
1890,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
1891,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
1892,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [58]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SMILES          1894 non-null   object
 1   IC50(microM)    1894 non-null   object
 2   database_label  1894 non-null   object
dtypes: object(3)
memory usage: 44.5+ KB


## Missing Values

In [59]:
combined_data.isnull().sum()

SMILES            0
IC50(microM)      0
database_label    0
dtype: int64

## Removing Duplicates

In [60]:
combined_data.iloc[:, 0].duplicated().sum()

6

In [61]:
combined_duplicates = combined_data.loc[combined_data.iloc[:, 0].duplicated(keep=False), :]
force_show_all(combined_duplicates)

,SMILES,IC50(microM),database_label
1739,NC1=CC2=NSN=C2C=C1C#C,2.11,Literature_1
1743,NC1=NC(N2CCC(C)(N)CC2)=CN=C1C3=CC=CC(Cl)=C3Cl,0.07,Literature_1
1748,O=C(CCN1C=C(N=N1)C(C2=C(C=C(C(C(O)=O)=C2)O)N3C)=C3C4=CC=CC=C4)NC(C=C5)=CC=C5C6=CC=CC=C6,5.5,Literature_1
1750,NC1=NC(N2CCC(C)(N)CC2)=CN=C1C3=CC=CC(Cl)=C3Cl,0.071,Literature_1
1769,O=C(CCN1C=C(N=N1)C(C2=C(C=C(C(C(O)=O)=C2)O)N3C)=C3C4=CC=CC=C4)NC(C=C5)=CC=C5C6=CC=CC=C6,5.5,Literature_1
1798,O=C1O[C@@]2(O)CC[C@@]3(C)[C@](CC[C@]3([H])[C@H](C)/C=C/[C@H](C)C(C)C)([H])C2=C1,6.75,Literature_1
1799,ClC1=C(Cl)C(C2=NC=C(N3CCC(N)(C)CC3)N=C2N)=CC=C1,0.07,Literature_2
1801,ClC1=CC=CC=C1CN2C3=NN=C(C4=C(O)C(OC)=CC=C4)N3C5=CC=CC=C5C2=O,60.0,Literature_2
1815,O=C1O[C@@]2(O)CC[C@@]3(C)[C@](CC[C@]3([H])[C@H](C)/C=C/[C@H](C)C(C)C)([H])C2=C1,0.02,Literature_2
1845,ClC1=C(Cl)C(C2=NC=C(N3CCC(N)(C)CC3)N=C2N)=CC=C1,0.07,Literature_2


In [62]:
combined_data.drop_duplicates(subset ="SMILES" ,keep='last',inplace=True)
combined_data

,SMILES,IC50(microM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
1889,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
1890,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
1891,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
1892,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [63]:
combined_data['database_label'].value_counts()

BindingDB       1738
Literature_2      93
Literature_1      57
Name: database_label, dtype: int64

## SMILES Validity

In [64]:
def check_smiles_validity(df):
    smiles = list(df['SMILES'].values)
    invalid_smile_indices = []
    invalid_chemistry_indices = []

    for idx, smile in enumerate(tqdm(smiles)):
        m = Chem.MolFromSmiles(smile, sanitize=False)
        if m is None:
            print(idx, smile, "invalid smile")
            invalid_smile_indices.append(idx)
        else:
            try:
                Chem.SanitizeMol(m)
            except:
                print(idx, smile, "invalid chemistry")
                invalid_chemistry_indices.append(idx)
  
    return invalid_smile_indices, invalid_chemistry_indices

In [65]:
invalid_smiles, invalid_chemistry = check_smiles_validity(combined_data)

  0%|          | 0/1888 [00:00<?, ?it/s]

626 Clc1ccc2O[V]3(=O)(Oc4ccc(Cl)cc4C=[N]3c3ccccc3)[N](=Cc2c1)c1ccccc1 invalid chemistry
647 [O-][N+](=O)c1ccc2O[V]3(=O)(Oc4ccc(cc4C=[N]3c3ccccc3)[N+]([O-])=O)[N](=Cc2c1)c1ccccc1 invalid chemistry
734 [OH2+][V]12([OH2+])(=O)OC(=O)c3ccccc3[N]1=Cc1cc(ccc1O2)[N+]([O-])=O invalid chemistry
1187 Brc1ccc2O[V]3(=O)(Oc4ccc(Br)cc4C=[N]3c3ccccc3)[N](=Cc2c1)c1ccccc1 invalid chemistry
1219 [OH2+][V]12([OH2+])(=O)OC(=O)c3ccccc3[N]1=Cc1cc(Br)ccc1O2 invalid chemistry
1269 ON(=O)c1cccc(N\N=C2/C(=O)Nc3ccc(cc23)S(O)(=O)=O)c1 invalid chemistry
1430 ON(=O)c1cccc(N\N=C2/C(=O)Nc3ccccc23)c1 invalid chemistry
1870 O=c(C1#NNC2=CC=C(S(=O)(O)=O)C=C2)n(C3=CC=CC=C3)N=C1C4=CC=C([N+]([O-])=O)C=C4 invalid chemistry
1878 O=C(C1CCN(C(C2=CC=CO2)=O)CC1)NC3=CC=C(C4=CC=C(NC(C5CCN(C(C6=CC=CO6)=O)CC5)#[OH])C=C4)C=C3 invalid chemistry


[23:56:00] Explicit valence for atom # 17 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 18 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 13 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 17 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 13 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 1 N, 4, is greater than permitted
[23:56:00] Explicit valence for atom # 1 N, 4, is greater than permitted
[23:56:01] Explicit valence for atom # 2 C, 5, is greater than permitted
[23:56:01] Explicit valence for atom # 25 C, 5, is greater than permitted


In [66]:
invalid_smiles

[]

In [67]:
invalid_chemistry

[626, 647, 734, 1187, 1219, 1269, 1430, 1870, 1878]

In [68]:
force_show_all(combined_data.loc[invalid_chemistry])

,SMILES,IC50(microM),database_label
626,Clc1ccc2O[V]3(=O)(Oc4ccc(Cl)cc4C=[N]3c3ccccc3)[N](=Cc2c1)c1ccccc1,2.3,BindingDB
647,[O-][N+](=O)c1ccc2O[V]3(=O)(Oc4ccc(cc4C=[N]3c3ccccc3)[N+]([O-])=O)[N](=Cc2c1)c1ccccc1,2.6,BindingDB
734,[OH2+][V]12([OH2+])(=O)OC(=O)c3ccccc3[N]1=Cc1cc(ccc1O2)[N+]([O-])=O,4.2,BindingDB
1187,Brc1ccc2O[V]3(=O)(Oc4ccc(Br)cc4C=[N]3c3ccccc3)[N](=Cc2c1)c1ccccc1,35.0,BindingDB
1219,[OH2+][V]12([OH2+])(=O)OC(=O)c3ccccc3[N]1=Cc1cc(Br)ccc1O2,49.0,BindingDB
1269,ON(=O)c1cccc(N\N=C2/C(=O)Nc3ccc(cc23)S(O)(=O)=O)c1,66.5,BindingDB
1430,ON(=O)c1cccc(N\N=C2/C(=O)Nc3ccccc23)c1,>100.0,BindingDB
1870,O=C1C(SC2=C(C(F)(F)F)N=CC=C2)=CN=C(N3CCC4(CO[C@@H](C)[C@H]4N)CC3)N1,0.031,Literature_2
1878,O=C(O)[C@H]([C@@]1([H])NC(C(O)=O)=C(C)CS1)NC([C@H](S(=O)(O)=O)C2=CC=CC=C2)=O,1.38,Literature_2


In [ ]:
# combined_data.loc[invalid_chemistry[0],'SMILES'] = r"Clc1cc2c(O[V]3(Oc4c(CN3c5ccccc5)cc(Cl)cc4)(N(c6ccccc6)C2)=O)cc1"
# combined_data.loc[invalid_chemistry[1],'SMILES'] = r"[O-][N+](c1cc2c(O[V]3(Oc4c(CN3c5ccccc5)cc([N+]([O-])=O)cc4)(N(c6ccccc6)C2)=O)cc1)=O"
# combined_data.loc[invalid_chemistry[2],'SMILES'] = r"[OH2+][V]12([OH2+])(OC(c3c(N1Cc4c(O2)ccc([N+]([O-])=O)c4)cccc3)=O)=O"
# combined_data.loc[invalid_chemistry[3],'SMILES'] = r"Brc1cc2c(O[V]3(Oc4c(CN3c5ccccc5)cc(Br)cc4)(N(c6ccccc6)C2)=O)cc1"
# combined_data.loc[invalid_chemistry[4],'SMILES'] = r"[OH2+][V]12([OH2+])(OC(c3ccccc3N1Cc4cc(Br)ccc4O2)=O)=O"
# combined_data.loc[invalid_chemistry[5],'SMILES'] = r"O=Nc1cccc(N/N=C2C(Nc3ccc(S(O)(=O)=O)cc3\2)=O)c1"
# combined_data.loc[invalid_chemistry[6],'SMILES'] = r"O=Nc1cccc(N/N=C2C(Nc3ccccc3\2)=O)c1"
# combined_data.loc[invalid_chemistry[7],'SMILES'] = r"O=c(C1=NNC2=CC=C(C=C2)S(=O)(O)=O)n(N=C1C3=CC=C(C=C3)[N+]([O-])=O)C4=CC=CC=C4"
# combined_data.loc[invalid_chemistry[8],'SMILES'] = r"O=C(NC1=CC=C(C=C1)C2=CC=C(C=C2)NC(C3CCN(CC3)C(C4=CC=CO4)=O)=O)C5CCN(CC5)C(C6=CC=CO6)=O"

In [69]:
def modify_dataframe(data, indices):
    data = data.drop(indices, axis=0)
    data = data.reset_index()
    data = data.drop(['index'],axis=1)
    return data

In [70]:
combined_data = modify_dataframe(combined_data, invalid_chemistry)
combined_data

,SMILES,IC50(microM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
1874,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
1875,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
1876,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
1877,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [71]:
combined_data.shape

(1879, 3)

In [72]:
invalid_smiles, invalid_chemistry = check_smiles_validity(combined_data)

  0%|          | 0/1879 [00:00<?, ?it/s]

1862 O=c(C1#NNC2=CC=C(S(=O)(O)=O)C=C2)n(C3=CC=CC=C3)N=C1C4=CC=C([N+]([O-])=O)C=C4 invalid chemistry
1869 O=C(C1CCN(C(C2=CC=CO2)=O)CC1)NC3=CC=C(C4=CC=C(NC(C5CCN(C(C6=CC=CO6)=O)CC5)#[OH])C=C4)C=C3 invalid chemistry


[23:56:34] Explicit valence for atom # 2 C, 5, is greater than permitted
[23:56:34] Explicit valence for atom # 25 C, 5, is greater than permitted


In [73]:
# combined_data.loc[invalid_chemistry[0],'SMILES'] = r'O=c(C1=NNC2=CC=C(C=C2)S(=O)(O)=O)n(N=C1C3=CC=C(C=C3)[N+]([O-])=O)C4=CC=CC=C4'
# combined_data.loc[invalid_chemistry[1],'SMILES'] = r'O=C(NC1=CC=C(C=C1)C2=CC=C(C=C2)NC(C3CCN(CC3)C(C4=CC=CO4)=O)=O)C5CCN(CC5)C(C6=CC=CO6)=O'

In [74]:
combined_data = modify_dataframe(combined_data, invalid_chemistry)
combined_data

,SMILES,IC50(microM),database_label
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008,BindingDB
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001,BindingDB
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001,BindingDB
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012,BindingDB
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015,BindingDB
...,...,...,...
1872,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64,Literature_2
1873,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58,Literature_2
1874,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47,Literature_2
1875,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36,Literature_2


In [75]:
invalid_smiles, invalid_chemistry = check_smiles_validity(combined_data)

  0%|          | 0/1877 [00:00<?, ?it/s]

In [76]:
combined_data.shape

(1877, 3)

## Calculating RDKit Descriptors

### 2D

In [79]:
class RDKit_2D:
    def __init__(self, smiles):
        self.mols = [Chem.MolFromSmiles(i) for i in smiles]
        self.smiles = smiles
        
        
    def compute_2Drdkit(self, name):
        rdkit_2d_desc = []
        calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
        header = calc.GetDescriptorNames()
        for i in tqdm(range(len(self.mols))):
            try:
                ds = calc.CalcDescriptors(self.mols[i])
                rdkit_2d_desc.append(ds)
            except:
                print(self.smiles[i])
        df = pd.DataFrame(rdkit_2d_desc,columns=header)
        df.insert(loc=0, column='SMILES', value=self.smiles)
        df.to_csv(name + '_RDKit_2D.csv', index=False)

In [93]:
def main():
    filename = "shp2_1877"         
    smiles = [i for i in combined_data['SMILES'].values]  
    ## Compute RDKit_2D Fingerprints and export a csv file.
    RDKit_descriptor = RDKit_2D(smiles)        # create your RDKit_2D object and provide smiles
    RDKit_descriptor.compute_2Drdkit(filename) # compute RDKit_2D and provide the name of your desired output file. you can use the same name as the input file because the RDKit_2D class will ensure to add "_RDKit_2D.csv" as part of the output file.

if __name__ == '__main__':
    main()

  0%|          | 0/1877 [00:00<?, ?it/s]

In [81]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [94]:
descriptor_data = pd.read_csv('shp2_1877_RDKit_2D.csv')
descriptor_data

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,12.992564,-0.169513,12.992564,0.068072,0.609173,455.389,431.197,454.132717,160,...,0,0,0,0,0,0,0,0,0,0
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,13.061568,-0.781987,13.061568,0.246174,0.340043,597.145,563.881,596.208486,216,...,0,0,0,0,0,0,0,0,0,0
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,12.340744,-0.676170,12.340744,0.017202,0.606557,485.419,459.211,484.154515,172,...,0,0,0,0,0,0,0,0,0,0
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,10.063495,-0.264210,10.063495,0.058620,0.603375,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,6.464779,0.086398,6.464779,0.086398,0.614971,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.277779,0.520256,6.277779,0.520256,0.882336,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,0
1873,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.275110,-0.097703,6.275110,0.097703,0.859533,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,0
1874,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,6.067947,0.568313,6.067947,0.568313,0.640083,277.331,262.211,277.132745,104,...,0,0,0,0,0,0,0,0,0,0
1875,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,9.796768,-0.064310,9.796768,0.064310,0.865831,366.292,345.124,365.106168,128,...,0,0,0,0,0,0,0,0,0,0


In [95]:
descriptor_data['IC50(microM)'] = combined_data['IC50(microM)']
descriptor_data.shape

(1877, 210)

In [96]:
descriptor_data.isnull().sum()

SMILES               0
MaxEStateIndex       0
MinEStateIndex       0
MaxAbsEStateIndex    0
MinAbsEStateIndex    0
                    ..
fr_thiocyan          0
fr_thiophene         0
fr_unbrch_alkane     0
fr_urea              0
IC50(microM)         0
Length: 210, dtype: int64

In [97]:
descriptor_data_na = descriptor_data[descriptor_data.isna().any(axis=1)]
descriptor_data_na

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50(microM)
1655,[O-][V]([O-])([O-])=O,8.609375,-5.875000,8.609375,5.87500,0.324875,114.938,114.938,113.928463,32,...,0,0,0,0,0,0,0,0,0,0.024
1822,O=C1C=CC2=C(C3=CC=CC=C3C(O)=O)C4=CC=C(O)C([As]...,13.194508,-1.742006,13.194508,0.01005,0.244942,664.515,646.371,663.846901,178,...,0,0,0,0,0,0,0,0,0,0.074


In [98]:
descriptor_data_na['SMILES'].values

array(['[O-][V]([O-])([O-])=O',
       'O=C1C=CC2=C(C3=CC=CC=C3C(O)=O)C4=CC=C(O)C([As]5SCCS5)=C4OC2=C1[As]6SCCS6'],
      dtype=object)

In [99]:
descriptor_data = descriptor_data.dropna()
descriptor_data

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50(microM)
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,12.992564,-0.169513,12.992564,0.068072,0.609173,455.389,431.197,454.132717,160,...,0,0,0,0,0,0,0,0,0,0.0008
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,13.061568,-0.781987,13.061568,0.246174,0.340043,597.145,563.881,596.208486,216,...,0,0,0,0,0,0,0,0,0,0.001
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,12.340744,-0.676170,12.340744,0.017202,0.606557,485.419,459.211,484.154515,172,...,0,0,0,0,0,0,0,0,0,0.001
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,10.063495,-0.264210,10.063495,0.058620,0.603375,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0.0012
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,6.464779,0.086398,6.464779,0.086398,0.614971,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0.0015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.277779,0.520256,6.277779,0.520256,0.882336,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,1.64
1873,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.275110,-0.097703,6.275110,0.097703,0.859533,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,1.58
1874,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,6.067947,0.568313,6.067947,0.568313,0.640083,277.331,262.211,277.132745,104,...,0,0,0,0,0,0,0,0,0,9.47
1875,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,9.796768,-0.064310,9.796768,0.064310,0.865831,366.292,345.124,365.106168,128,...,0,0,0,0,0,0,0,0,0,1.36


In [100]:
training_molecules = descriptor_data[['SMILES','IC50(microM)']]
training_molecules

,SMILES,IC50(microM)
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,0.0008
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,0.001
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,0.001
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,0.0012
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,0.0015
...,...,...
1872,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.64
1873,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,1.58
1874,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,9.47
1875,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,1.36


In [101]:
training_molecules.to_csv('SHP2_training_1875.csv',index=False)

In [102]:
descriptor_data

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50(microM)
0,Cc1nc(cc(=O)n1-c1cccc(Cl)c1Cl)N1CCC2(Cc3ccccc3...,12.992564,-0.169513,12.992564,0.068072,0.609173,455.389,431.197,454.132717,160,...,0,0,0,0,0,0,0,0,0,0.0008
1,N[C@@H]1CCCC11CCN(CC1)c1cnc(Sc2cccc(NC(=O)c3c(...,13.061568,-0.781987,13.061568,0.246174,0.340043,597.145,563.881,596.208486,216,...,0,0,0,0,0,0,0,0,0,0.001
2,N[C@@H]1c2ccccc2CC11CCN(CC1)c1nc(N)c(C2=C(Cl)C...,12.340744,-0.676170,12.340744,0.017202,0.606557,485.419,459.211,484.154515,172,...,0,0,0,0,0,0,0,0,0,0.001
3,N[C@@H]1C[C@H](O)CC11CCN(CC1)c1ncc(Sc2cccc(Cl)...,10.063495,-0.264210,10.063495,0.058620,0.603375,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0.0012
4,C[C@@H]1OCC2(CCN(CC2)c2ncc(Sc3cccc(Cl)c3Cl)c3n...,6.464779,0.086398,6.464779,0.086398,0.614971,464.422,441.238,463.100037,158,...,0,0,0,0,0,0,0,0,0,0.0015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,NC1=C(N2CCC(CN)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.277779,0.520256,6.277779,0.520256,0.882336,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,1.64
1873,NC1=C(N2CCC(C)(N)CC2)N=CC(C3=CC=CC(Cl)=C3Cl)=C1,6.275110,-0.097703,6.275110,0.097703,0.859533,351.281,331.121,350.106502,122,...,0,0,0,0,0,0,0,0,0,1.58
1874,NC1=C(NC2=CC(N)=CC=C2)N=CC(C3=CC=CN=C3)=C1,6.067947,0.568313,6.067947,0.568313,0.640083,277.331,262.211,277.132745,104,...,0,0,0,0,0,0,0,0,0,9.47
1875,NCC(CC1)CCN1C2=C(CO)C=C(C3=CC=CC(Cl)=C3Cl)C=N2,9.796768,-0.064310,9.796768,0.064310,0.865831,366.292,345.124,365.106168,128,...,0,0,0,0,0,0,0,0,0,1.36


In [103]:
descriptor_data.to_csv('SHP2_train_descriptors_1875.csv',index=False)